# Applied Data Science Capstone Project (Week 3)
## Segmenting and Clustering Neighborhoods in Toronto

In [2]:
# Import modules
import numpy as np
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cmap
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [8]:
# Scrape data from Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_alldata = pd.read_html(url, header=0)[0]
df_alldata.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
#Remove Boroughs that are 'Not assigned'
df_canada = df_alldata[df_alldata.Borough != 'Not assigned'].reset_index(drop = True)
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
# combine neighborhoods in same postal code in one row with the neighborhoods separated with commas
df_canada["Neighbourhood"] = df_canada.groupby("Postal Code")["Neighbourhood"].transform(lambda ii: ', '.join(ii))

#remove duplicates
df_canada = df_canada.drop_duplicates()
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
# cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
if sum(df_canada["Neighbourhood"] == "Not assigned")==0:
    print("No neighborhood with \"Not assigned\" status")
else:
    for idx, row in df_canada.iterrows():
        if row["Neighbourhood"] == "Not assigned":
            df_canada.at[idx, "Neighbourhood"] == row["Borough"]
    print("Filled Neighborhood info")

No neighborhood with "Not assigned" status


In [19]:
# print DataFrame shape
df_canada.shape

(103, 3)